In [2]:
import math
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from datetime import datetime
from gensim.models import Word2Vec

warnings.filterwarnings('ignore')

In [10]:
train = pd.read_csv('./data/train.txt', header=None, names=['pid', 'label', 'gender', 'age', 'tagid', 'time', 'province', 'city', 'make', 'model'])
test = pd.read_csv('./data/apply_new.txt', header=None, names=['pid', 'gender', 'age', 'tagid', 'time', 'province', 'city', 'make', 'model'])
data = pd.concat([train, test])

In [11]:
data['tagid'] = data['tagid'].apply(lambda x: eval(x))
sentences = data['tagid'].values.tolist()
for i in range(len(sentences)):
    sentences[i] = [str(x) for x in sentences[i]]

emb_size = 32
model = Word2Vec(sentences, vector_size=emb_size, window=6, min_count=5, sg=0, hs=0, seed=1, epochs=5)

emb_matrix = []
for seq in sentences:
    vec = []
    for w in seq:
        if w in model.wv:
            vec.append(model.wv.get_vector(w))
    if len(vec) > 0:
        emb_matrix.append(np.mean(vec, axis=0))
    else:
        emb_matrix.append([0] * emb_size)
emb_matrix = np.array(emb_matrix)
for i in range(emb_size):
    data['tag_emb_{}'.format(i)] = emb_matrix[:, i]

In [16]:
cat_cols = ['gender', 'age', 'province', 'city']
features = [i for i in data.columns if i not in ['pid', 'label', 'tagid', 'time', 'model', 'make']]

data[cat_cols] = data[cat_cols].astype('category')
X_train = data[~data['label'].isna()]
X_test = data[data['label'].isna()]

y = X_train['label']
KF = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
params = {
          'objective':'binary',
          'metric':'binary_error',
          'learning_rate':0.05,
          'subsample':0.8,
          'subsample_freq':3,
          'colsample_btree':0.8,
          'num_iterations': 10000,
          'verbose':-1
}
oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros((len(X_test)))
# 特征重要性
feat_imp_df = pd.DataFrame({'feat': features, 'imp': 0})
# 五折交叉验证
for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y.values)):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    trn_data = lgb.Dataset(X_train.iloc[trn_idx][features],label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx][features],label=y.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(
        params,
        trn_data,
        num_round,
        valid_sets = [trn_data, val_data],
        verbose_eval=100,
        early_stopping_rounds=50,
        categorical_feature=cat_cols,
    )
    feat_imp_df['imp'] += clf.feature_importance() / 5
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(X_test[features], num_iteration=clf.best_iteration)
print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

fold n°0
trn_idx: [     0      1      2 ... 299997 299998 299999]
val_idx: [     3      9     14 ... 299992 299993 299994]
Training until validation scores don't improve for 50 rounds
[100]	training's binary_error: 0.290596	valid_1's binary_error: 0.312317
[200]	training's binary_error: 0.273738	valid_1's binary_error: 0.309217
[300]	training's binary_error: 0.263133	valid_1's binary_error: 0.308483
Early stopping, best iteration is:
[308]	training's binary_error: 0.262408	valid_1's binary_error: 0.3076
fold n°1
trn_idx: [     0      1      2 ... 299996 299998 299999]
val_idx: [    10     13     25 ... 299986 299988 299997]
Training until validation scores don't improve for 50 rounds
[100]	training's binary_error: 0.292612	valid_1's binary_error: 0.311617
[200]	training's binary_error: 0.2748	valid_1's binary_error: 0.308333
Early stopping, best iteration is:
[217]	training's binary_error: 0.272587	valid_1's binary_error: 0.3074
fold n°2
trn_idx: [     0      3      5 ... 299997 299998

In [20]:
X_test['category_id'] = [1 if i >= 2.5 else 0 for i in predictions_lgb]
X_test['user_id'] = X_test['pid']
X_test[['user_id', 'category_id']].to_csv('base_wv.csv', index=False)

In [24]:
feat_imp_df.sort_values(by='imp', ascending=False)

,feat,imp
3,city,4694.4
32,tag_emb_28,212.8
33,tag_emb_29,178.0
11,tag_emb_7,167.6
12,tag_emb_8,166.4
10,tag_emb_6,161.4
22,tag_emb_18,153.2
29,tag_emb_25,151.2
25,tag_emb_21,147.4
35,tag_emb_31,146.8
